In [3]:
import csv
import json
import time
import pandas as pd

In [4]:
df = pd.read_csv("data/input.csv")

In [8]:
df

,timestamp,day,priceType,pnodeName,hour,lmp,mcc,mlc
0,2026-01-02 00:00:00,2026-01-02,ExPost,MEC.MECB,0,41.76,-3.08,0.38
1,2026-01-02 00:00:00,2026-01-02,ExPost,CWLP.AZ,0,38.35,-4.90,-4.76
2,2026-01-02 00:00:00,2026-01-02,ExPost,ILLINOIS.HUB,0,36.32,-1.63,-1.54
3,2026-01-02 00:00:00,2026-01-02,ExPost,PSI_GEN.AGG,0,33.87,0.76,0.44
4,2026-01-02 00:00:00,2026-01-02,ExPost,MINN.HUB,0,35.17,1.25,-0.33
...,...,...,...,...,...,...,...,...
1435,2026-01-02 23:55:00,2026-01-02,ExPost,MEC.MECB,23,41.00,-0.48,3.73
1436,2026-01-02 23:55:00,2026-01-02,ExPost,CWLP.AZ,23,36.84,-5.76,-0.68
1437,2026-01-02 23:55:00,2026-01-02,ExPost,ILLINOIS.HUB,23,36.09,-0.91,-0.89
1438,2026-01-02 23:55:00,2026-01-02,ExPost,PSI_GEN.AGG,23,35.93,-1.11,0.03


In [5]:
hourly = (
    df.groupby(["day", "priceType", "pnodeName", "hour"], as_index=False)
      .agg({"lmp": "mean", "mcc": "mean", "mlc": "mean"})
      .round(2)
)

In [6]:
hourly

,day,priceType,pnodeName,hour,lmp,mcc,mlc
0,2026-01-02,ExPost,CWLP.AZ,0,37.33,-1.11,-1.07
1,2026-01-02,ExPost,CWLP.AZ,1,35.59,-0.64,-1.73
2,2026-01-02,ExPost,CWLP.AZ,2,35.75,-1.29,-1.21
3,2026-01-02,ExPost,CWLP.AZ,3,34.43,-0.94,-2.11
4,2026-01-02,ExPost,CWLP.AZ,4,35.13,-1.96,-1.08
...,...,...,...,...,...,...,...
115,2026-01-02,ExPost,PSI_GEN.AGG,19,36.55,-0.80,-1.83
116,2026-01-02,ExPost,PSI_GEN.AGG,20,35.71,-1.20,-1.75
117,2026-01-02,ExPost,PSI_GEN.AGG,21,36.01,-1.68,-0.58
118,2026-01-02,ExPost,PSI_GEN.AGG,22,34.67,-1.37,-1.69


In [10]:
for (day, price_type, timestamp), g_min in df.groupby(
    ["day", "priceType", "timestamp"]
):
    pnodes = []

    for pnode, g_pnode in g_min.groupby("pnodeName"):
        row = g_pnode.iloc[0]

        pnodes.append({
            "pnodeName": pnode,
            "hours": [{
                "hour": int(row["hour"]),
                "lmp": float(row["lmp"]),
                "mcc": float(row["mcc"]),
                "mlc": float(row["mlc"])
            }]
        })

    payload = {
        "day": day,
        "priceType": price_type,
        "timestamp": timestamp,
        "pnodes": pnodes
    }
    print(json.dumps(payload))

{"day": "2026-01-02", "priceType": "ExPost", "timestamp": "2026-01-02 00:00:00", "pnodes": [{"pnodeName": "CWLP.AZ", "hours": [{"hour": 0, "lmp": 38.35, "mcc": -4.9, "mlc": -4.76}]}, {"pnodeName": "ILLINOIS.HUB", "hours": [{"hour": 0, "lmp": 36.32, "mcc": -1.63, "mlc": -1.54}]}, {"pnodeName": "MEC.MECB", "hours": [{"hour": 0, "lmp": 41.76, "mcc": -3.08, "mlc": 0.38}]}, {"pnodeName": "MINN.HUB", "hours": [{"hour": 0, "lmp": 35.17, "mcc": 1.25, "mlc": -0.33}]}, {"pnodeName": "PSI_GEN.AGG", "hours": [{"hour": 0, "lmp": 33.87, "mcc": 0.76, "mlc": 0.44}]}]}
{"day": "2026-01-02", "priceType": "ExPost", "timestamp": "2026-01-02 00:05:00", "pnodes": [{"pnodeName": "CWLP.AZ", "hours": [{"hour": 0, "lmp": 38.2, "mcc": -1.1, "mlc": -1.96}]}, {"pnodeName": "ILLINOIS.HUB", "hours": [{"hour": 0, "lmp": 34.3, "mcc": 1.45, "mlc": -1.89}]}, {"pnodeName": "MEC.MECB", "hours": [{"hour": 0, "lmp": 38.85, "mcc": -0.26, "mlc": -3.9}]}, {"pnodeName": "MINN.HUB", "hours": [{"hour": 0, "lmp": 35.91, "mcc": -0.